# 1 Architekturkonzept: Chatbot für Berliner Gesetze (Open Source)

Ziel ist ein webbasiertes Softwaresystem, das aktuell gültige Berliner Gesetze und Verordnungen aus der Quelle *gesetze.berlin.de* automatisiert verarbeitet, versioniert und Verwaltungsmitarbeitende über einen Chatbot beim Auffinden und Verstehen relevanter Rechtsgrundlagen unterstützt.

# 2 Entwicklungsschritte als Minimum Viable Produkt (MVP)-Roadmap

1. Problemdefinition, Zielsetzung, Definition der Ziele bzw. Rahmenbedingungen, Zielgruppe. </br>

2. Strategie & Priorisierung der Kernfunktionen vom Chatbot, Annahmen.</br>

3. Design & Prototyping:</br>
   - Architekturkonzept erarbeiten inkl. Architekturskizze, beispielsweise als Mermaid.</br>
   - Datenflüsse setzen.</br>
   - Open Source Technologien definieren.</br>
   - Risiken evaluieren und Mitigationsstrategie.</br>

4. Entwicklung eines Pilots.</br>

5. Ausrollen & Feedback: Veröffentlichung bei frühen Nutzern, Sammeln von Daten (KPIs) und direktem Feedback (Umfrage).</br>

7. Iteration: Nutzung des Feedbacks, um das Produkt schrittweise zu verbessern und die Roadmap für die nächste Version anzupassen. Debugging. </br>


# 3 Frontend Lösungsvorschläge


## 3.1 Variante A - Chatbot Wigdet
Der Chatbot wird als “Widget” per JavaScript-Snippet implementiert, wodurch ein kleines Script in die Webanwendung einzubinden ist.
Das Frontend lädt beispielsweise das Script *chat-widget.js*, ein Widget bestehend aus Button plus Chatfenster wird eingebunden. Die Nutzer-Nachrichten (Anfragen) gehen per HTTPS Protokoll (REST Architektur) oder WebSocket an das Backend, um die Ressourcen von *https://gesetze.berlin.de/bsbe/search* abzurufen.

**Vorteil:** Einfach zu integrieren in bestehende Seiten, wenig Coupling. Geeignet für eine erste Entwicklunsphase, um den Bedarf der Nutzer zu messen.</br>
**Nachteil:** Authentifizierungsmethode wie Auth/SSO (Token ans Widget) sollen einwandfrei gelöst werden.

## 3.2 Variante B - “Native” Integration

Der Chatbot wird als UI-Komponente im bestehenden Frontend (kein iFrame, kein externes Widget) direkt integriert. In der Regel wird dies als (React/Angular) umgesetzt.</br>

**Vorteil:** Konsistentes Design & UX, direkte Kommunikation mit der eigenen Chat API, direkter Zugriff auf Anwendungskontext (Nutzerrolle, Berechtigungen), bessere Performance & Stabilität.</br>
**Nachteil:** Höherer Implementierungsaufwand.

# 4 Implementierung (nach MVP)

## 4.1 Problemdefinition

Technische Architektur eines webbasierten Softwaresystems bzw. eines Chatbots, der zur Unterstützung bei der Recherche von Rechtsgrundlagen in Berliner Gesetze und Verordnungen per Chatbot dient.


## 4.2 Strategie & Priorisierung der Kernfunktionen vom Chatbot

**Funktionale Anforderungen an die Applicaction (Kernfunktionen)**
- Der Chatbot muss nur aktuelle gültige Berliner Gesetze und Verordnungen automatisch verarbeitet.
- Der Chatbot soll die deutsche Sprache (native) unterstützen. Typische Verwaltungsfragen sind auf Deutsch, Webseite (Quelle) ebenfalls auf Deutsch aufgebaut.
- Der Chatbot soll als Umfang der Antworten mindestens Zitaten von Quellen (§/Absatz) und Link zum Dokument liefern. 
- Der Chatbot soll Feedback-Funktion enthalten (z.B.: 👍/👎, „Quelle unpassend“, „Antwort unklar“).
- Der Chatbot soll keine Rechtsberatung bieten, es ist nur für Assistenz und Recherche gedacht.
- Der Chatbot darf nocht “live” bei jeder Nutzerfrage auf *gesetze.berlin.de* zugreifen, sondern auf einen eigenen, lokal regelmäßig gepflegten Wissensspeicher.


**Nicht-funktionale Anforderungen an die Applicaction (Kernfunktionen)**
- Der Chatbot soll möglichst schnelle Reaktionszeit (Angabe fehlt!) zeigen, um hohe Nutzerakzeptanz zu erzielen.

**Annahmen und Rahmenbedingungen**
- Die Inhalte von *gesetze.berlin.de* (Berliner Vorschriften- und Rechtsprechungsdatenbank) sind öffentlich zugängliche (HTML-Seiten).
- Für die Anwendungen dürfen ausschließlich nur Open-Source-Komponente verwendet werden.
- Das Chatbot-System ist intern für Verwaltungsmitarbeitende gedacht, eventuell für ein Behörden-/Intranet-Use Case gedacht: SSO, rollenbasierter Zugriff.
- Bezüglich Datenschutz: Nutzer sollen keine personenbezogenen Falldaten in den Chat kopieren. Ebenfalls kein Copy-Paste ganzer Akten, Schutz vor Missbrauch.
- Der Limit nach Tokens per Nutzeranfrage (Frontend soft-Limit) ist UX-getrieben. Beispielsweise darf der Nutzer maximal 5.000 Zeichen eintippen (1 Token ≈ 3–4 Zeichen im Deutschen (grob)). Backend hard-Limit wird auf 8.000 Zeichen gesetzt.
- Es wird auf LLM-Inference verzichten, Chatbot nur für Retrieval und Antwortvorlagen konzipiert.
- In einer ersten Implementierungsphase ist das Hauptziel, Digitalisierung mithilfe von modernen webbasierten Applikationen innerhalb der Senatsverwaltung für Justiz und Verbraucherschutz voranzutreiben. Dabei dürfte die Akzeptanz der Nutzer, nämlich die Verwaltungsmitarbeitende, zu der Applikation nach dem Ausrollen hoch sein.

## 4.3 Design & Prototyping

### 4.3.1 Architektureskizze

Quelle: Entworfen in *diagrams.net*.

![xx](./berlin-laws-chatbot-architecture.drawio.png)

### 4.3.2 Verarbeitungsschritte


1. Ein Scheduler startet regelmäßig den Importprozess von Inhalten und sichert, dass andere Komponenten nach definierter Reihenfolge und Abhängigkeiten laufen (Crawler/Fetcher, Parser, SQL-DB, Index, etc.). Zeitgesteuerte Datenerfassung (täglich/mehrmals täglich). 

2. Der Crawler/Fetcher ruft neue oder geänderte Inhalte von *gesetze.berlin.de* ab. Er sucht nach Änderungen und lädt diese Inhalte schonend (Rate Limit, Retry, Cache) herunter (HTML-Seiten). 

3. Die abgerufenen Dokumente bzw. Rohdaten (HTML/PDF?) werden intern im Object Store versioniert gespeichert. Sämtliche nachgelagerten Verarbeitungsschritte (Parsing, Indexierung, Chat API) greifen ausschließlich auf diese interne Kopie zu.

4. Ein Parser extrahiert aus den Rohdaten den eigentlichen Gesetzestext, erkennt die Gesetzesstruktur (z. B. §, Titel, Absatz, Nummerierung) und extrahiert die Metadaten (Stand, Links, Gültigkeit). Die Texte werden bereinigt und standardisiert für Indexierung. Das Ergebnis sind strukturierte, normalisierte, maschinenlesbare Text-Chunks (Beispiel: JSON) mit Metadaten. 

5. Die strukturierte Text-Chunks und deren Metadaten werden in einer SQL-Datenbank gespeichert. Sommit werden persistent auditierbare Daten zur Verfügung gestellt, als auditierbare “Single Source of Truth”.

6. Die Text-Chunks werden in einen Search-Index überführt, suchrelevante Felder (Text, §, Stand) werden indexiert. Der Search Index nimmt die Nutzerfrage entgegen, findet relevante Paragraphen, sortiert sie nach Relevanz, liefert kurze Textausschnitte (Snippets) und übergibt diese an die Chat API.

7. Ein authentifizierter Nutzer stellt eine Frage über das Web-Interface (Chat-Fenster). Die Chat API validiert die Anfrage und Berechtigungen (Auth/SSO), dann eine gültige, verarbeitbare Anfrage liegt vor.

8. Die Chat API (Backend) stellt eine Suchanfrage an den Search-Index. Die relevantesten Text-Chunks werden ermittelt, Gültigkeit und Metadaten werden über die SQL-Datenbank verifiziert.

9. Die Antworten-Treffer werden zu einer nutzerfreundlichen Antwort zusammengestellt und in Form von Snippets und Quellenangaben (Dokument, §, Absatz) an das Frontend (Chat-Fenster) zurückgegeben.


### 4.3.3 Systemkomponenten (Open Source)

Note: Die konkrete Auswahl hängt von Betriebsumgebung, Team‑Skills und Skalierungsbedarf ab.

- **Scheduler & Orchestrator:** Apache Airflow.
- **Crawler/Fetcher:** Scrapy (robust).
- **Object Store:** MinIO.
- **Parser & Normalizer:** BeautifulSoup.
- **SQL-DB:** PostgreSQL.
- **Search Index:** OpenSearch (BM25/Volltextsuche).
- **Vector DB (optional):** findet passende Absätze, auch wenn Wörter in der Nutzerfrage nicht exakt übereinstimmen. Ohne Vektor Suche, wird dies nur nach Volltext geführt. Tool: Qdrant.
- **Frontend:** Chat UI als Widget OR Iframe OR Native, siehe Kapitel 3.
- **Chat API (Backend):** FastAPI in Python.
- **Auth / SSO:** Keycloak.
- **Observability (optional):** Grafana.

### 4.3.4 Risiken und Herausforderungen

**Rechtliche Compliance**
- Nutzungs-/Lizenzfragen beim automatisierten Abruf und der Weiterverarbeitung (auch wenn Inhalte öffentlich sind), Datenschutz prüfen, ggf. Abstimmung mit Betreiberstelle. 
- Rate Limits: Crawler darf Portal nicht belasten. 

**Qualität**
- Halluzinationen: Im Fall der Nutzung von LLMs.
- Falscher Rechtsstand: Gesetzt hat sich geändert und noch nicht aktualisiert worden, “gültig ab/bis”.
- Kontextverständnis: Verwaltungssprache und Abkürzungen, Querverweise zwischen Normen.

**Security**
- Prompt Injection: User versucht Systemregeln auszuhebeln, im Fall der Nutzung von LLMs.
- Datenabfluss: falls Nutzer interne Fallinformationen eintippt.

**Betrieb/Skalierung**
- Hohe Latenz: Caching wichtig.
- Software Update bzw. Rebuilds: Deployment neuer Softwarenversionen soll einwandfrei laufen, getrennte Umgebungen (Dev/Test/Prod).
- Monitoring & Fehlerfälle: Beispiel: Änderungen in der  HTML-Struktur der Quelle verursacht das Scheitern vom Parser. 

**Akzeptanz/UX**
- Niedrige Nutzenakzeptanz aufgrund von hoher Latenz, nicht zutreffenden Antworten, unklare Hinweise vom Chatbot, etc.

## 4.4 Ausrollen & Feedback (KPIs)

**KPIs zur Messung der Nutzerakzeptanz:**
- Nutzung: aktive Users, Messages pro Session, etc.
- Erfolg: Task Completion Rate, Anzahl der Antworten als hilfreich markiert, % der Anfragen ohne passende Treffer, etc.
- Vertrauen: Thumbs-Up / Thumbs-Down Ratio, % der gemeldeten Fehler, etc.
- Effizienzgewinn: Anteil der Chats statt klassischer Suche, etc.

Bei Bedarf lässt sich dafür ein interaktives Dashboarding in Grafana aufbauen.

## 4.5 Iteration

Neue Software Dependencies, Versionen und Releases, Prgrammerungsprache-Bibliotheken, neue Chatbot Features, Debugging, etc. führen zu einer Weiterentwicklung und Verbesserung des Chatbots. Dies soll möglichst versioniert und gut kommentiert sein. Zu beachten: Change Managemnent (CM), neue MVPs, Chatbot-Softwarelösung containerisiert in Docker betreiben, etc.

# 5 Follow-up

Weitere Ideen:
- Integration von KI-Fähigkeiten, falls Normen und Datenschutz konform.